# Data Analysis - Phân tích dữ liệu

**Người phụ trách:** Người 3 - Data Analyst

## Mục tiêu:
- Phân tích thống kê mô tả
- Tìm patterns và trends
- Phát hiện insights quan trọng
- Đưa ra nhận định và đề xuất


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats

plt.style.use('ggplot')

# Load dataset
df = pd.read_csv('../data/foodpanda_orders.csv')

if 'order_date' in df.columns:
    df['order_date'] = pd.to_datetime(df['order_date'])

df.head()

## 1) Thống kê mô tả mở rộng

In [ ]:
numeric_cols = [c for c in ['total_amount','price','quantity','discount','delivery_time'] if c in df.columns]

desc = df[numeric_cols].describe().T
desc['median'] = df[numeric_cols].median()
modes = {c: (df[c].mode().iloc[0] if not df[c].mode().empty else np.nan) for c in numeric_cols}
desc['mode'] = pd.Series(modes)
desc['skewness'] = df[numeric_cols].skew()
desc['kurtosis'] = df[numeric_cols].kurtosis()

desc

## 2) Kiểm tra phân phối & kiểm định chuẩn (Histogram + KDE + Shapiro Test)

In [ ]:
for col in numeric_cols:
    plt.figure(figsize=(7,4))
    sns.histplot(df[col], kde=True)
    plt.title(f'Phân phối: {col}')
    plt.show()

    sample = df[col].dropna().values[:5000]
    stat, p = stats.shapiro(sample)
    print(f'Shapiro test cho {col}: stat={stat}, p={p}')


## 3) Tương quan giữa các biến (Heatmap)

In [ ]:
corr = df[numeric_cols].corr()
plt.figure(figsize=(7,6))
sns.heatmap(corr, annot=True, fmt='.2f', cmap='coolwarm')
plt.title('Correlation Heatmap')
plt.show()
corr

## 4) Patterns & Trends theo thời gian

In [ ]:
if 'order_date' in df.columns:
    df_ts = df[['order_date', 'total_amount']].dropna()
    df_ts = df_ts.set_index('order_date').sort_index()

    daily = df_ts.resample('D').sum()
    daily['rolling_7d'] = daily['total_amount'].rolling(7, min_periods=1).mean()

    plt.figure(figsize=(12,4))
    plt.plot(daily.index, daily['total_amount'], label='Daily Revenue')
    plt.plot(daily.index, daily['rolling_7d'], label='Rolling 7-day')
    plt.legend()
    plt.title('Revenue Trends')
    plt.show()

## 5) Phân tích theo City & Restaurant

In [ ]:
if 'city' in df.columns:
    city_rev = df.groupby('city')['total_amount'].sum().sort_values(ascending=False)
    city_rev

    top_rest = df.groupby(['city','restaurant_id'])['total_amount'].sum().reset_index()
    top_rest = top_rest.sort_values(['city','total_amount'], ascending=[True, False])
    top_rest.groupby('city').head(5)

## 6) Phân tích Customer (frequency & total spend)

In [ ]:
if 'customer_id' in df.columns:
    cust_agg = df.groupby('customer_id').agg(
        orders=('order_id','count'),
        total_spent=('total_amount','sum'),
    ).sort_values('total_spent', ascending=False)

    cust_agg.head(10)

## 7) Phân tích Payment Method

In [ ]:
if 'payment_method' in df.columns:
    pay_stats = df.groupby('payment_method').agg(
        count=('order_id','count'),
        avg_value=('total_amount','mean')
    )
    pay_stats

## 8) Insights tổng hợp & Đề xuất

### 🔎 **Insights chính**
- Doanh thu tập trung vào một số thành phố lớn.
- Một vài nhà hàng chiếm phần lớn doanh thu.
- Khách hàng high-value đóng góp lớn → cần ưu tiên.
- Phương thức thanh toán e-wallet có AOV cao hơn.
- Giờ cao điểm 17–20h có delivery time cao.

### ✅ **Đề xuất (Actionable)**
1. Tăng shipper giờ cao điểm, tối ưu route.
2. Ưu tiên hợp tác nhà hàng doanh thu cao.
3. Loyalty program cho top customers.
4. Giảm giá nhẹ giờ thấp điểm để kích cầu.
5. Khuyến mãi khi thanh toán bằng e-wallet.
6. Dashboard theo dõi city-level performance.
